# Control drone flight using keyboard

Control flight of tello drone using keyboard inputs.

Import libraries

In [ ]:
import pygame  # keyboard input handling
import djitellopy as tello 
from time import sleep
import threading
import cv2  # Import OpenCV for video frame operations

Function to return value of pressed key

In [2]:
def get_pressed_key():
    """Return the name of the key pressed, or None if no key is pressed."""
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            return pygame.key.name(event.key)
        else:
            return None

Function to translate pressed key into flight command

In [3]:
def translate_pressed_key(my_key):
    """translate keys into drone motions"""

    # move up/down and turn right/left
    if my_key == "left":
        my_drone.rotate_clockwise(90)
    if my_key == "right":
        my_drone.rotate_counter_clockwise(90)
    if my_key == "up":
        my_drone.move_up(20)
    if my_key == "down":
        my_drone.move_down(20)

    # move forward/back
    if my_key == "f":
        my_drone.move_forward(20)
    if my_key == "a":
        my_drone.move_back(20)

    # land
    if my_key == "q": # land = q
        my_drone.end()

Function to control flight routine

In [4]:
def flight_routine(my_drone):
    print("waiting for event to signal video stream readiness")
    stream_ready.wait()

    print("event signaled for video stream readiness")
    
    # control take off
    running = True
    while running:
        my_key = get_pressed_key()
        print(my_key)
        if my_key == "e":  #
            my_drone.takeoff()
            running = False
        sleep(0.05)  # sleep for 50 ms

    # control flight program with keys
    running = True
    while running:
        my_key = get_pressed_key()
        print(my_key)
        translate_pressed_key(my_key)
        sleep(0.05)  # sleep for 50 ms
        if my_key == "q":
            running = False


Function to get sensor data

In [5]:
def get_status_data(my_drone):
    """Acquire other status data and return as dictionary"""
    status_data = {
        'flight_time':my_drone.get_flight_time(),
        'battery':my_drone.get_battery(),
        'height':my_drone.get_height(),
        'tof':my_drone.get_distance_tof(),
        'temph':my_drone.get_highest_temperature(),
        'barometer':my_drone.get_barometer()
    }
    return(status_data)

Function to control vide streaming

In [6]:
def stream_video(my_drone):
    while True:
    # get sensor data 
        my_status_data = get_status_data(my_drone)

        frame = my_drone.get_frame_read().frame

        # print sensor data to screen
        cv2.putText(frame, "Battery " + str(my_status_data['battery']) + " percent", (10, 20), 
                cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 234, 255), 1,2)
        
        cv2.imshow('tello stream', frame)

        # Check if streaming ready. 
        if not stream_ready.is_set():

            # Signal that video streaming is ready
            stream_ready.set()
            print("Event Signal Set: Stream is live.")

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    cv2.destroyAllWindows()

## Main program

In [ ]:
# initialize pygame modules (for keyboard input)
pygame.init() 
window = pygame.display.set_mode((400, 400))

# create event object from threading module (object = module.function())
stream_ready = threading.Event()

# Initialize my_drone as tello object, connect to it, and turn its video stream on.
my_drone = tello.Tello() # object = instance.Class()
my_drone.connect()
my_drone.streamon()
print("drone connected and stream on. Starting video stream thread.\n")

# set stream_video as thread target
stream_thread = threading.Thread(target=stream_video, args=(my_drone,))

# Set thread as a daemon and start it
stream_thread.daemon = True
stream_thread.start()

# Execute the flight routine
flight_routine(my_drone)

print("Flight routine ended. Rebooting drone.")

# Reboot the drone at the end
drone.reboot()

pygame.quit()